### Relax Inc. mock takehome challenge
Task: Defining  an  "adopted  user"   as  a  user  who   has  logged  into  the  product  on  three  separate
days  in  at  least  one  seven­day  period ,  identify  which  factors  predict  future  user
adoption .

In [28]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from datetime import datetime

#### Read files

In [2]:
engagement_file = "takehome_user_engagement.csv"
users_file = "takehome_users2.csv"

In [50]:
engagement = pd.DataFrame(pd.read_csv(engagement_file, parse_dates=True))
users = pd.DataFrame(pd.read_csv(users_file, parse_dates=True, encoding = 'latin1'))

In [30]:
users[:3]

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,4/22/2014 3:53,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,11/15/2013 3:45,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,3/19/2013 23:14,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0


In [32]:
engagement[:3]

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1


In [51]:
engagement['time_stamp'] = engagement.time_stamp.apply(lambda d: datetime.strptime(d, '%Y-%m-%d %H:%M:%S'))

In [52]:
engagement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
time_stamp    207917 non-null datetime64[ns]
user_id       207917 non-null int64
visited       207917 non-null int64
dtypes: datetime64[ns](1), int64(2)
memory usage: 4.8 MB


In [53]:
engagement.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [ ]:
engagement['date']=

#### Adopted users
Identify which users have logged into the product on 3 days in one 7-day period.
<br>
use engagement table -- ```user_id``` == ```object_id```

Strategy: 
- count visits per day by user
- set non-zero == 1
- use rolling window sum and look for sum > 7

In [15]:
engagement.day=engagement.time_stamp.apply(lambda d: d.day)

AttributeError: 'str' object has no attribute 'day'

In [14]:
grouped = engagement.groupby(['user_id','time_stamp'])[['visited']].sum()
#grouped['totalRents'] = bike2015_2016.groupby(['Year','timeDelt'])['timeDelt'].size()
grouped.head()

visited
user_id time_stamp                  
1       2014-04-22 03:53:30        1
2       2013-11-15 03:45:04        1
        2013-11-29 03:45:04        1
        2013-12-09 03:45:04        1
        2013-12-25 03:45:04        1